In [1]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import requests

# load image from the IAM database
url = 'text_blocks/text_block_12.png'
image = Image.open(url).convert("RGB")

processor = TrOCRProcessor.from_pretrained('microsoft/trocr-large-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-large-handwritten')
pixel_values = processor(images=image, return_tensors="pt").pixel_values

generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
generated_text

/home/tpittal/.conda/envs/ads/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 24,
  "patch_size": 16,
  "qkv_bias": false,
  "transformers_version": "4.47.0"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "a

'Wreckolas Edwards . Instead to pursue them'

In [15]:
pip install transformers

  Using cached huggingface_hub-0.26.3-py3-none-any.whl.metadata (13 kB)
  Using cached regex-2024.11.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached safetensors-0.4.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 69.6 MB/s eta 0:00:00
Using cached huggingface_hub-0.26.3-py3-none-any.whl (447 kB)
Using cached regex-2024.11.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (781 kB)
Using cached safetensors-0.4.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (435 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 73.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
import cv2
import numpy as np
import os
from PIL import Image

# Load the image
input_path = "dupickens_a-1_042_binary_otsu.png"
output_dir = "data/split_images/"  # Directory to save the split images
os.makedirs(output_dir, exist_ok=True)

# Read the image
image = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)

# Apply binary thresholding
_, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Sum pixel values along the horizontal axis to create a horizontal projection profile
horizontal_projection = np.sum(binary_image, axis=1)

# Detect regions with text based on the projection profile
threshold = np.max(horizontal_projection) * 0.1  # Adjust sensitivity as needed
line_indices = np.where(horizontal_projection > threshold)[0]

# Group consecutive indices to detect lines
lines = []
current_line = [line_indices[0]]
for i in range(1, len(line_indices)):
    if line_indices[i] - line_indices[i - 1] > 1:  # New line detected
        lines.append(current_line)
        current_line = [line_indices[i]]
    else:
        current_line.append(line_indices[i])
lines.append(current_line)

# Crop and save each detected line
for idx, line in enumerate(lines):
    y_min = min(line)
    y_max = max(line)
    
    # Add a small margin (optional)
    margin = 5
    y_min = max(0, y_min - margin)
    y_max = min(image.shape[0], y_max + margin)
    
    # Crop the line
    cropped_line = image[y_min:y_max, :]
    
    # Save the cropped line
    line_path = os.path.join(output_dir, f"line_{idx + 1}.png")
    cv2.imwrite(line_path, cropped_line)

print(f"Image split into {len(lines)} lines and saved to {output_dir}")


Image split into 83 lines and saved to data/split_images/


In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

# Define directories
input_dir = "output/test"  # Directory containing the original images
output_dir = "processed_texts"  # Directory to save OCR results
os.makedirs(output_dir, exist_ok=True)

# Load the TrOCR model and processor
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-large-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-large-handwritten')

# Filter files with '_binary_otsu.png' in their names
input_files = [f for f in os.listdir(input_dir) if f.endswith('_binary_otsu.png')]

# Process each image
for input_file in input_files:
    input_path = os.path.join(input_dir, input_file)
    base_name = os.path.splitext(input_file)[0]  # e.g., dupickens_a-1_011
    image_output_dir = os.path.join(output_dir, base_name)  # Create a folder for this image's text
    os.makedirs(image_output_dir, exist_ok=True)
    
    # Read the image
    image = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
    
    # Apply binary thresholding
    _, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Sum pixel values along the horizontal axis to create a horizontal projection profile
    horizontal_projection = np.sum(binary_image, axis=1)
    
    # Detect regions with text based on the projection profile
    threshold = np.max(horizontal_projection) * 0.1  # Adjust sensitivity as needed
    line_indices = np.where(horizontal_projection > threshold)[0]
    
    # Group consecutive indices to detect lines
    lines = []
    current_line = [line_indices[0]]
    for i in range(1, len(line_indices)):
        if line_indices[i] - line_indices[i - 1] > 1:  # New line detected
            lines.append(current_line)
            current_line = [line_indices[i]]
        else:
            current_line.append(line_indices[i])
    lines.append(current_line)
    
    # Process each detected line
    for idx, line in enumerate(lines):
        y_min = min(line)
        y_max = max(line)
        
        # Add a small margin (optional)
        margin = 5
        y_min = max(0, y_min - margin)
        y_max = min(image.shape[0], y_max + margin)
        
        # Crop the line
        cropped_line = image[y_min:y_max, :]
        
        # Save the cropped line temporarily
        temp_line_path = f"{image_output_dir}/line_{idx + 1}.png"
        cv2.imwrite(temp_line_path, cropped_line)
        
        # OCR Processing
        line_image = Image.open(temp_line_path).convert("RGB")
        pixel_values = processor(images=line_image, return_tensors="pt").pixel_values
        generated_ids = model.generate(pixel_values)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        
        # Save OCR text for this line
        line_text_path = os.path.join(image_output_dir, f"line_{idx + 1}.txt")
        with open(line_text_path, 'w') as text_file:
            text_file.write(generated_text)
    
    print(f"Processed {input_file}: {len(lines)} lines saved to {image_output_dir}")


Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 24,
  "patch_size": 16,
  "qkv_bias": false,
  "transformers_version": "4.46.2"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 1024,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decod

Processed dupickens_a-1_051_binary_otsu.png: 67 lines saved to processed_texts/dupickens_a-1_051_binary_otsu
Processed dupickens_a-1_052_binary_otsu.png: 76 lines saved to processed_texts/dupickens_a-1_052_binary_otsu
Processed dupickens_a-1_053_binary_otsu.png: 80 lines saved to processed_texts/dupickens_a-1_053_binary_otsu


In [1]:
pip install torch torchvision opencv-python


  Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (62.5 MB)
Using cached fsspec-2024.10.0-py3-none-any.whl (179 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install paddleocr paddlepaddle-gpu


  Using cached paddleocr-2.9.1-py3-none-any.whl.metadata (8.5 kB)
  Using cached shapely-2.0.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.0 kB)
  Using cached scikit_image-0.24.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (14 kB)
  Using cached imgaug-0.4.0-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached pyclipper-1.3.0.post6-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (9.0 kB)
  Using cached lmdb-1.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.1 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached rapidfuzz-3.10.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached Cython-3.0.11-cp310-cp31

In [5]:
from paddleocr import PaddleOCR, draw_ocr
import cv2
from PIL import Image

# Initialize PaddleOCR with the DBNet++ detection model
ocr = PaddleOCR(det_model_dir='path_to_dbnet++_det_model', use_gpu=False)

# Path to input image
image_path = "dupickens_a-1_041_original.png"

# Run OCR for text detection
results = ocr.ocr(image_path, det=True, rec=False)

# Load image for visualization
image = Image.open(image_path).convert('RGB')
boxes = [line[0] for line in results[0]]

# Draw detected text boxes on the image
image_with_boxes = draw_ocr(image, boxes, txts=None, scores=None, font_path='path_to_font.ttf')

# Save and display the result
result_image_path = "detected_text_result.jpg"
image_with_boxes.save(result_image_path)
print(f"Detection result saved at {result_image_path}")


[2024/12/05 23:36:29] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='path_to_dbnet++_det_model', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/home/tpittal/.paddleocr/whl/rec/ch/ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='/

AttributeError: 'numpy.ndarray' object has no attribute 'save'

In [13]:
from paddleocr import PaddleOCR, draw_ocr
import cv2
from PIL import Image

# Initialize PaddleOCR with the DBNet++ detection model
ocr = PaddleOCR(
    det_model_dir='path_to_dbnet++_det_model',
    det_db_box_thresh=0.1,  # Lower the box threshold to detect faint text
    det_db_unclip_ratio=3,  # Increase unclip ratio to detect tightly packed text
    use_gpu=False
    )

# Path to input image
image_path = "dupickens_a-1_042_binary_otsu.png"

# Run OCR for text detection
results = ocr.ocr(image_path, det=True, rec=True)

# Load image for visualization
image = Image.open(image_path).convert('RGB')
boxes = [line[0] for line in results[0]]

# Draw detected text boxes on the image
image_with_boxes = draw_ocr(image, boxes, txts=None, scores=None, font_path='path_to_font.ttf')

# Convert the numpy.ndarray result to a Pillow Image
image_with_boxes = Image.fromarray(image_with_boxes)

# Save and display the result
result_image_path = "detected_text_result.jpg"
image_with_boxes.save(result_image_path)
print(f"Detection result saved at {result_image_path}")


[2024/12/05 23:43:25] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='path_to_dbnet++_det_model', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.1, det_db_unclip_ratio=3, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/home/tpittal/.paddleocr/whl/rec/ch/ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='/ho